In [ ]:
!pip install datasets

In [ ]:
!pip install torch==2.0.1
!pip install transformers[torch] accelerate -U

# Twitter Model com teste dados youtube

In [ ]:
import os
import numpy as np
import pandas as pd
from datasets import Dataset, load_metric
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Parâmetros
MODEL_NAME = "rufimelo/Legal-BERTimbau-large"
DIMENSION_SIZE = 512
SEED = 42
MODEL_PATH = '../Datasets/Twitter/final_model'
TRAIN_SIZE = 0.8
VALIDATION_SIZE = 0.1
TEST_SIZE = 0.1
# Função para calcular as métricas de desempenho
def compute_metrics(eval_pred):
    accuracy = load_metric("accuracy", trust_remote_code=True)
    precision = load_metric("precision", trust_remote_code=True)
    recall = load_metric("recall", trust_remote_code=True)
    f1 = load_metric("f1", trust_remote_code=True)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    prec = precision.compute(predictions=predictions, references=labels, average=None)
    rec = recall.compute(predictions=predictions, references=labels, average=None)
    f1_score = f1.compute(predictions=predictions, references=labels, average=None)

    metrics_per_class = {f"precision_class_{i}": p for i, p in enumerate(prec["precision"])}
    metrics_per_class.update({f"recall_class_{i}": r for i, r in enumerate(rec["recall"])})
    metrics_per_class.update({f"f1_class_{i}": f for i, f in enumerate(f1_score["f1"])})

    return metrics_per_class

# Função para carregar e preparar os dados
def prepare_data(file_path):
    df = pd.read_csv(file_path, sep='\t')
    df['comentario'] = df['comentario'].apply(str)
    df['comentario'] = df['comentario'].apply(lambda x: x.replace('\n', ' ').replace('\t', ' ').strip())
    df['rotulo_manual'] = df['rotulo_manual'].apply(lambda x: 2 if x == -1 else x)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    dataset = Dataset.from_pandas(df)
    return dataset

# Função para carregar e preparar os dados
# def prepare_data(file_path, train_size, validation_size, test_size):
#     df = pd.read_csv(file_path, sep='\t')
#     df['comentario'] = df['comentario'].apply(str)
#     df['comentario'] = df['comentario'].apply(lambda x: x.replace('\n', ' ').replace('\t', ' ').strip())
#     df['rotulo'] = df['rotulo'].apply(lambda x: 2 if x == -1 else x)
#     df = df.sample(frac=1).reset_index(drop=True)

#     print(df.groupby("rotulo").count())

#     train_val_df, test_df = train_test_split(df, test_size=test_size, stratify=df['rotulo'], random_state=SEED)
#     train_df, val_df = train_test_split(train_val_df, test_size=validation_size/(train_size + validation_size), stratify=train_val_df['rotulo'], random_state=SEED)

#     train_dataset = Dataset.from_pandas(train_df)
#     val_dataset = Dataset.from_pandas(val_df)
#     test_dataset = Dataset.from_pandas(test_df)

#     return train_dataset, val_dataset, test_dataset

# Tokenização dos dados
def tokenize_and_format(examples):
    tokenized_inputs = tokenizer(examples['comentario'], padding='max_length', max_length=DIMENSION_SIZE, truncation=True)
    tokenized_inputs['labels'] = examples['rotulo']
    return tokenized_inputs

# Carregar o modelo e o tokenizador salvos
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

# Preparar os argumentos de treinamento (necessário para Trainer, mesmo que não vá treinar novamente)
training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=8,  # Pode ajustar o batch size de acordo com sua capacidade
    logging_dir='./logs',
)

# Carregar e preparar a nova base de dados
new_data_path = '../Datasets/YouTube/4mil_consistentes_preprocessado.csv'  # Substitua pelo caminho da nova base de dados
train_dataset, val_dataset, test_dataset = prepare_data(new_data_path)
new_dataset = new_dataset.map(tokenize_and_format, batched=True)

# Criar um Trainer para realizar a inferência
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
)

# Realizar a inferência
predictions_metrics = trainer.predict(new_dataset)
predictions_new_data = np.argmax(predictions_metrics.predictions, axis=-1)
labels_new_data = np.array(new_dataset['labels'])
new_data_metrics = compute_metrics((predictions_metrics.predictions, labels_new_data))

print("New Data Metrics:", new_data_metrics)

# Salvando as previsões e rótulos para uso posterior
np.savez('../Datasets/Twitter vs Youtube/tt_model_yt_data_predictions_and_labels.npz', predictions=predictions_new_data, labels=labels_new_data)


In [ ]:
# Parâmetros do bootstrap
BOOTSTRAP_ITERATIONS = 5000
BOOTSTRAP_SEED = 2024

# Função para realizar o bootstrap no conjunto de teste
def bootstrap_test(predictions, labels, n_iterations=BOOTSTRAP_ITERATIONS):
    rng = np.random.default_rng(BOOTSTRAP_SEED)
    idx = np.arange(labels.shape[0])

    precision = load_metric("precision", trust_remote_code=True)
    recall = load_metric("recall", trust_remote_code=True)
    f1 = load_metric("f1", trust_remote_code=True)

    precision_scores_class_0, recall_scores_class_0, f1_scores_class_0 = [], [], []
    precision_scores_class_1, recall_scores_class_1, f1_scores_class_1 = [], [], []
    precision_scores_class_2, recall_scores_class_2, f1_scores_class_2 = [], [], []

    for _ in range(n_iterations):
        pred_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        pred_boot = predictions[pred_idx]
        labels_boot = labels[pred_idx]

        precision_class_0 = precision.compute(predictions=pred_boot, references=labels_boot, average=None)["precision"][0]
        recall_class_0 = recall.compute(predictions=pred_boot, references=labels_boot, average=None)["recall"][0]
        f1_class_0 = f1.compute(predictions=pred_boot, references=labels_boot, average=None)["f1"][0]

        precision_class_1 = precision.compute(predictions=pred_boot, references=labels_boot, average=None)["precision"][1]
        recall_class_1 = recall.compute(predictions=pred_boot, references=labels_boot, average=None)["recall"][1]
        f1_class_1 = f1.compute(predictions=pred_boot, references=labels_boot, average=None)["f1"][1]

        precision_class_2 = precision.compute(predictions=pred_boot, references=labels_boot, average=None)["precision"][2]
        recall_class_2 = recall.compute(predictions=pred_boot, references=labels_boot, average=None)["recall"][2]
        f1_class_2 = f1.compute(predictions=pred_boot, references=labels_boot, average=None)["f1"][2]

        precision_scores_class_0.append(precision_class_0)
        recall_scores_class_0.append(recall_class_0)
        f1_scores_class_0.append(f1_class_0)

        precision_scores_class_1.append(precision_class_1)
        recall_scores_class_1.append(recall_class_1)
        f1_scores_class_1.append(f1_class_1)

        precision_scores_class_2.append(precision_class_2)
        recall_scores_class_2.append(recall_class_2)
        f1_scores_class_2.append(f1_class_2)

    ci_metrics = {}
    for metric_name, scores in zip(
        ["precision_class_0", "recall_class_0", "f1_class_0",
         "precision_class_1", "recall_class_1", "f1_class_1",
         "precision_class_2", "recall_class_2", "f1_class_2"],
        [precision_scores_class_0, recall_scores_class_0, f1_scores_class_0,
         precision_scores_class_1, recall_scores_class_1, f1_scores_class_1,
         precision_scores_class_2, recall_scores_class_2, f1_scores_class_2]
    ):
        ci_metrics[metric_name] = {
            "mean": np.mean(scores),
            "std": np.std(scores),
            "lower_95_ci": np.percentile(scores, 2.5),
            "upper_95_ci": np.percentile(scores, 97.5)
        }

    return ci_metrics

# Carregar as previsões e rótulos salvos
data = np.load('../Datasets/Twitter vs Youtube/tt_model_yt_data_predictions_and_labels.npz')
predictions_test = data['predictions']
labels_test = data['labels']

# Bootstrapping no conjunto de teste
ci_metrics = bootstrap_test(predictions_test, labels_test, n_iterations=BOOTSTRAP_ITERATIONS)

# Salvando os resultados do intervalo de confiança
results = pd.DataFrame(ci_metrics).T
results.to_csv('../Datasets/Twitter vs Youtube/tt_model_yt_data_bootstrap_results.csv')

print("Média, Desvio Padrão e Intervalos de Confiança das Métricas para as classes:")
print(results)

# Youtube Model com teste dados twitter

In [ ]:
import os
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Parâmetros
MODEL_NAME = "rufimelo/Legal-BERTimbau-large"
DIMENSION_SIZE = 512
MODEL_PATH = '../Datasets/YouTube/final_model'

# Função para calcular as métricas de desempenho
def compute_metrics(eval_pred):
    accuracy = load_metric("accuracy", trust_remote_code=True)
    precision = load_metric("precision", trust_remote_code=True)
    recall = load_metric("recall", trust_remote_code=True)
    f1 = load_metric("f1", trust_remote_code=True)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    prec = precision.compute(predictions=predictions, references=labels, average=None)
    rec = recall.compute(predictions=predictions, references=labels, average=None)
    f1_score = f1.compute(predictions=predictions, references=labels, average=None)

    metrics_per_class = {f"precision_class_{i}": p for i, p in enumerate(prec["precision"])}
    metrics_per_class.update({f"recall_class_{i}": r for i, r in enumerate(rec["recall"])})
    metrics_per_class.update({f"f1_class_{i}": f for i, f in enumerate(f1_score["f1"])})

    return metrics_per_class

# Função para carregar e preparar os dados
def prepare_data(file_path):
    df = pd.read_csv(file_path, sep='\t')
    df['comentario'] = df['comentario'].apply(str)
    df['comentario'] = df['comentario'].apply(lambda x: x.replace('\n', ' ').replace('\t', ' ').strip())
    df['rotulo_manual'] = df['rotulo_manual'].apply(lambda x: 2 if x == -1 else x)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    dataset = Dataset.from_pandas(df)
    return dataset

# Função para carregar e preparar os dados
# def prepare_data(file_path, train_size, validation_size, test_size):
#     df = pd.read_csv(file_path, sep='\t')
#     df['comentario'] = df['comentario'].apply(str)
#     df['comentario'] = df['comentario'].apply(lambda x: x.replace('\n', ' ').replace('\t', ' ').strip())
#     df['rotulo'] = df['rotulo'].apply(lambda x: 2 if x == -1 else x)
#     df = df.sample(frac=1).reset_index(drop=True)

#     print(df.groupby("rotulo").count())

#     train_val_df, test_df = train_test_split(df, test_size=test_size, stratify=df['rotulo'], random_state=SEED)
#     train_df, val_df = train_test_split(train_val_df, test_size=validation_size/(train_size + validation_size), stratify=train_val_df['rotulo'], random_state=SEED)

#     train_dataset = Dataset.from_pandas(train_df)
#     val_dataset = Dataset.from_pandas(val_df)
#     test_dataset = Dataset.from_pandas(test_df)

#     return train_dataset, val_dataset, test_dataset

# Tokenização dos dados
def tokenize_and_format(examples):
    tokenized_inputs = tokenizer(examples['comentario'], padding='max_length', max_length=DIMENSION_SIZE, truncation=True)
    tokenized_inputs['labels'] = examples['rotulo_manual']
    return tokenized_inputs

# Carregar o modelo e o tokenizador salvos
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

# Preparar os argumentos de treinamento (necessário para Trainer, mesmo que não vá treinar novamente)
training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=8,  # Pode ajustar o batch size de acordo com sua capacidade
    logging_dir='./logs',
)

# Carregar e preparar a nova base de dados
new_data_path = '../Datasets/Twitter/resultados_certos_marcos_BERT.csv'  # Substitua pelo caminho da nova base de dados
new_dataset = prepare_data(new_data_path)
new_dataset = new_dataset.map(tokenize_and_format, batched=True)

# Criar um Trainer para realizar a inferência
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
)

# Realizar a inferência
predictions_metrics = trainer.predict(new_dataset)
predictions_new_data = np.argmax(predictions_metrics.predictions, axis=-1)
labels_new_data = np.array(new_dataset['labels'])
new_data_metrics = compute_metrics((predictions_metrics.predictions, labels_new_data))

print("New Data Metrics:", new_data_metrics)

# Salvando as previsões e rótulos para uso posterior
np.savez('../Datasets/Twitter vs Youtube/yt_model_tt_data_predictions_and_labels.npz', predictions=predictions_new_data, labels=labels_new_data)


In [ ]:
from datasets import load_metric, Dataset
import os
import numpy as np
import pandas as pd
# Parâmetros do bootstrap
BOOTSTRAP_ITERATIONS = 5000
BOOTSTRAP_SEED = 2000

# Função para realizar o bootstrap no conjunto de teste
def bootstrap_test(predictions, labels, n_iterations=BOOTSTRAP_ITERATIONS):
    rng = np.random.default_rng(BOOTSTRAP_SEED)
    idx = np.arange(labels.shape[0])

    precision = load_metric("precision", trust_remote_code=True)
    recall = load_metric("recall", trust_remote_code=True)
    f1 = load_metric("f1", trust_remote_code=True)

    precision_scores_class_0, recall_scores_class_0, f1_scores_class_0 = [], [], []
    precision_scores_class_1, recall_scores_class_1, f1_scores_class_1 = [], [], []
    precision_scores_class_2, recall_scores_class_2, f1_scores_class_2 = [], [], []

    for _ in range(n_iterations):
        pred_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        pred_boot = predictions[pred_idx]
        labels_boot = labels[pred_idx]

        precision_class_0 = precision.compute(predictions=pred_boot, references=labels_boot, average=None)["precision"][0]
        recall_class_0 = recall.compute(predictions=pred_boot, references=labels_boot, average=None)["recall"][0]
        f1_class_0 = f1.compute(predictions=pred_boot, references=labels_boot, average=None)["f1"][0]

        precision_class_1 = precision.compute(predictions=pred_boot, references=labels_boot, average=None)["precision"][1]
        recall_class_1 = recall.compute(predictions=pred_boot, references=labels_boot, average=None)["recall"][1]
        f1_class_1 = f1.compute(predictions=pred_boot, references=labels_boot, average=None)["f1"][1]

        precision_class_2 = precision.compute(predictions=pred_boot, references=labels_boot, average=None)["precision"][2]
        recall_class_2 = recall.compute(predictions=pred_boot, references=labels_boot, average=None)["recall"][2]
        f1_class_2 = f1.compute(predictions=pred_boot, references=labels_boot, average=None)["f1"][2]

        precision_scores_class_0.append(precision_class_0)
        recall_scores_class_0.append(recall_class_0)
        f1_scores_class_0.append(f1_class_0)

        precision_scores_class_1.append(precision_class_1)
        recall_scores_class_1.append(recall_class_1)
        f1_scores_class_1.append(f1_class_1)

        precision_scores_class_2.append(precision_class_2)
        recall_scores_class_2.append(recall_class_2)
        f1_scores_class_2.append(f1_class_2)

    ci_metrics = {}
    for metric_name, scores in zip(
        ["precision_class_0", "recall_class_0", "f1_class_0",
         "precision_class_1", "recall_class_1", "f1_class_1",
         "precision_class_2", "recall_class_2", "f1_class_2"],
        [precision_scores_class_0, recall_scores_class_0, f1_scores_class_0,
         precision_scores_class_1, recall_scores_class_1, f1_scores_class_1,
         precision_scores_class_2, recall_scores_class_2, f1_scores_class_2]
    ):
        ci_metrics[metric_name] = {
            "mean": np.mean(scores),
            "std": np.std(scores),
            "lower_95_ci": np.percentile(scores, 2.5),
            "upper_95_ci": np.percentile(scores, 97.5)
        }

    return ci_metrics

# Carregar as previsões e rótulos salvos
data = np.load('../Datasets/Twitter vs Youtube/yt_model_tt_data_predictions_and_labels.npz')
predictions_test = data['predictions']
labels_test = data['labels']

# Bootstrapping no conjunto de teste
ci_metrics = bootstrap_test(predictions_test, labels_test, n_iterations=BOOTSTRAP_ITERATIONS)

# Salvando os resultados do intervalo de confiança
results = pd.DataFrame(ci_metrics).T
results.to_csv('../Datasets/Twitter vs Youtube/yt_model_tt_data_bootstrap_results.csv')

print("Média, Desvio Padrão e Intervalos de Confiança das Métricas para as classes:")
print(results)

# RF_tt com teste dados youtube

In [ ]:
!pip install datasets

In [ ]:
import os
import numpy as np
import pandas as pd
from datasets import load_metric
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import joblib

In [ ]:
# carrega matriz de embeddings dos dados de treino, teste e validação
train_embeddings = joblib.load('../Datasets/YoutubeRF_SVM/train_youtube.pkl')
val_embeddings = joblib.load('../Datasets/YoutubeRF_SVM/val_youtube.pkl')
test_embeddings = joblib.load('../Datasets/YoutubeRF_SVM/test_youtube.pkl')

y_train = joblib.load('../Datasets/YoutubeRF_SVM/y_train_youtube.pkl')
y_val = joblib.load('../Datasets/YoutubeRF_SVM/y_val_youtube.pkl')
y_test = joblib.load('../Datasets/YoutubeRF_SVM/y_test_youtube.pkl')

In [ ]:
# Carregando os modelos
best_rf_tt = joblib.load('../Datasets/TwitterRF_SVM/RF_best_model_twitter.pkl')


In [ ]:
# Avaliar o modelo no conjunto de teste e salvar previsões e rótulos
y_test_pred = best_rf_tt.predict(test_embeddings)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_class_report = classification_report(y_test, y_test_pred, target_names=['neutro', 'aprova', 'desaprova'])

print("Test Accuracy:", test_accuracy)
print("Test Classification Report:\n", test_class_report)


np.savez('../Datasets/TwitterRF_SVM/test_predictions_and_labels_rf_tt_model_testdata_youtube.npz', predictions=y_test_pred, labels=y_test)


In [ ]:
# Parâmetros do bootstrap
BOOTSTRAP_ITERATIONS = 5000
BOOTSTRAP_SEED = 2024
# Função para realizar o bootstrap no conjunto de teste
def bootstrap_test(predictions, labels, n_iterations=BOOTSTRAP_ITERATIONS):
    rng = np.random.default_rng(BOOTSTRAP_SEED)
    idx = np.arange(labels.shape[0])

    precision = load_metric("precision", trust_remote_code=True)
    recall = load_metric("recall", trust_remote_code=True)
    f1 = load_metric("f1", trust_remote_code=True)

    precision_scores_class_0, recall_scores_class_0, f1_scores_class_0 = [], [], []
    precision_scores_class_1, recall_scores_class_1, f1_scores_class_1 = [], [], []
    precision_scores_class_2, recall_scores_class_2, f1_scores_class_2 = [], [], []

    for _ in range(n_iterations):
        pred_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        pred_boot = predictions[pred_idx]
        labels_boot = labels[pred_idx]

        precision_class_0 = precision.compute(predictions=pred_boot, references=labels_boot, average=None)["precision"][0]
        recall_class_0 = recall.compute(predictions=pred_boot, references=labels_boot, average=None)["recall"][0]
        f1_class_0 = f1.compute(predictions=pred_boot, references=labels_boot, average=None)["f1"][0]

        precision_class_1 = precision.compute(predictions=pred_boot, references=labels_boot, average=None)["precision"][1]
        recall_class_1 = recall.compute(predictions=pred_boot, references=labels_boot, average=None)["recall"][1]
        f1_class_1 = f1.compute(predictions=pred_boot, references=labels_boot, average=None)["f1"][1]

        precision_class_2 = precision.compute(predictions=pred_boot, references=labels_boot, average=None)["precision"][2]
        recall_class_2 = recall.compute(predictions=pred_boot, references=labels_boot, average=None)["recall"][2]
        f1_class_2 = f1.compute(predictions=pred_boot, references=labels_boot, average=None)["f1"][2]

        precision_scores_class_0.append(precision_class_0)
        recall_scores_class_0.append(recall_class_0)
        f1_scores_class_0.append(f1_class_0)

        precision_scores_class_1.append(precision_class_1)
        recall_scores_class_1.append(recall_class_1)
        f1_scores_class_1.append(f1_class_1)

        precision_scores_class_2.append(precision_class_2)
        recall_scores_class_2.append(recall_class_2)
        f1_scores_class_2.append(f1_class_2)

    ci_metrics = {}
    for metric_name, scores in zip(
        ["precision_class_0", "recall_class_0", "f1_class_0",
         "precision_class_1", "recall_class_1", "f1_class_1",
         "precision_class_2", "recall_class_2", "f1_class_2"],
        [precision_scores_class_0, recall_scores_class_0, f1_scores_class_0,
         precision_scores_class_1, recall_scores_class_1, f1_scores_class_1,
         precision_scores_class_2, recall_scores_class_2, f1_scores_class_2]
    ):
        ci_metrics[metric_name] = {
            "mean": np.mean(scores),
            "std": np.std(scores),
            "lower_95_ci": np.percentile(scores, 2.5),
            "upper_95_ci": np.percentile(scores, 97.5)
        }

    return ci_metrics

In [ ]:
# Carregar as previsões e rótulos salvos
data_rf = np.load('../Datasets/TwitterRF_SVM/test_predictions_and_labels_rf_tt_model_testdata_youtube.npz')
predictions_test_rf = data_rf['predictions']
labels_test_rf = data_rf['labels']

In [ ]:
# Bootstrapping no conjunto de teste para RandomForest
ci_metrics_rf = bootstrap_test(predictions_test_rf, labels_test_rf, n_iterations=BOOTSTRAP_ITERATIONS)

In [ ]:
# Salvando os resultados do intervalo de confiança
results_rf = pd.DataFrame(ci_metrics_rf).T
results_rf.to_csv('../Datasets/TwitterRF_SVM/bootstrap_results_rf_tt_model_data_youtube.csv')

print("Random Forest - Média, Desvio Padrão e Intervalos de Confiança das Métricas para as classes:")
print(results_rf)

# RF_yt com teste dados twitter

In [ ]:
best_rf_yt = joblib.load('../Datasets/YoutubeRF_SVM/RF_best_model_youtube.pkl')

In [ ]:
# carrega matriz de embeddings dos dados de treino, teste e validação
train_embeddings = joblib.load('../Datasets/TwitterRF_SVM/train_youtube.pkl')
val_embeddings = joblib.load('../Datasets/TwitterRF_SVM/val_youtube.pkl')
test_embeddings = joblib.load('../Datasets/TwitterRF_SVM/test_youtube.pkl')


y_train = joblib.load('../Datasets/TwitterRF_SVM/y_train_youtube.pkl')
y_val = joblib.load('../Datasets/TwitterRF_SVM/y_val_youtube.pkl')
y_test = joblib.load('../Datasets/TwitterRF_SVM/y_test_youtube.pkl')

In [ ]:
# Avaliar o modelo no conjunto de teste e salvar previsões e rótulos
y_test_pred = best_rf_yt.predict(test_embeddings)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_class_report = classification_report(y_test, y_test_pred, target_names=['neutro', 'aprova', 'desaprova'])

print("Test Accuracy:", test_accuracy)
print("Test Classification Report:\n", test_class_report)


np.savez('../Datasets/YoutubeRF_SVM/test_predictions_and_labels_rf_yt_model_data_twitter.npz', predictions=y_test_pred, labels=y_test)


In [ ]:
# Carregar as previsões e rótulos salvos
data_rf = np.load('../Datasets/YoutubeRF_SVM/test_predictions_and_labels_rf_yt_model_data_twitter.npz')
predictions_test_rf = data_rf['predictions']
labels_test_rf = data_rf['labels']

In [ ]:
# Bootstrapping no conjunto de teste para RandomForest
ci_metrics_rf = bootstrap_test(predictions_test_rf, labels_test_rf, n_iterations=BOOTSTRAP_ITERATIONS)

In [ ]:
# Salvando os resultados do intervalo de confiança
results_rf = pd.DataFrame(ci_metrics_rf).T
results_rf.to_csv('../Datasets/YoutubeRF_SVM/bootstrap_results_rf_yt_model_data_twitter.csv')

print("Random Forest - Média, Desvio Padrão e Intervalos de Confiança das Métricas para as classes:")
print(results_rf)

# SVM_tt com teste dados youtube

In [ ]:
best_svm_tt = joblib.load('../Datasets/TwitterRF_SVM/SVM_best_model_twitter.pkl')

In [ ]:
# carrega matriz de embeddings dos dados de treino, teste e validação
train_embeddings = joblib.load('../Datasets/YoutubeRF_SVM/train_youtube.pkl')
val_embeddings = joblib.load('../Datasets/YoutubeRF_SVM/val_youtube.pkl')
test_embeddings = joblib.load('../Datasets/YoutubeRF_SVM/test_youtube.pkl')

y_train = joblib.load('../Datasets/YoutubeRF_SVM/y_train_youtube.pkl')
y_val = joblib.load('../Datasets/YoutubeRF_SVM/y_val_youtube.pkl')
y_test = joblib.load('../Datasets/YoutubeRF_SVM/y_test_youtube.pkl')

In [ ]:
# Normalizar os embeddings para o SVM
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
val_embeddings_scaled = scaler.transform(val_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

In [ ]:
# Avaliar o modelo no conjunto de teste e salvar previsões e rótulos
y_test_pred = best_svm_tt.predict(test_embeddings_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_class_report = classification_report(y_test, y_test_pred, target_names=['neutro', 'aprova', 'desaprova'])

print("Test Accuracy:", test_accuracy)
print("Test Classification Report:\n", test_class_report)


np.savez('../Datasets/TwitterRF_SVM/test_predictions_and_labels_svm_tt_model_testdata_youtube.npz', predictions=y_test_pred, labels=y_test)


In [ ]:
# Carregar as previsões e rótulos salvos
data_svm = np.load('../Datasets/TwitterRF_SVM/test_predictions_and_labels_svm_tt_model_testdata_youtube.npz')
predictions_test_svm = data_svm['predictions']
labels_test_svm = data_svm['labels']

In [ ]:
# Bootstrapping no conjunto de teste para RandomForest
ci_metrics_rf = bootstrap_test(predictions_test_svm, labels_test_svm, n_iterations=BOOTSTRAP_ITERATIONS)

In [ ]:
# Salvando os resultados do intervalo de confiança
results_rf = pd.DataFrame(ci_metrics_rf).T
results_rf.to_csv('../Datasets/TwitterRF_SVM/bootstrap_results_svm_tt_model_data_youtube.csv')

print("Random Forest - Média, Desvio Padrão e Intervalos de Confiança das Métricas para as classes:")
print(results_rf)

# SVM_yt com teste dados twitter

In [ ]:
best_svm_yt = joblib.load('../Datasets/YoutubeRF_SVM/SVM_best_model_youtube.pkl')


In [ ]:
# carrega matriz de embeddings dos dados de treino, teste e validação
train_embeddings = joblib.load('../Datasets/TwitterRF_SVM/train_youtube.pkl')
val_embeddings = joblib.load('../Datasets/TwitterRF_SVM/val_youtube.pkl')
test_embeddings = joblib.load('../Datasets/TwitterRF_SVM/test_youtube.pkl')

y_train = joblib.load('../Datasets/TwitterRF_SVM/y_train_youtube.pkl')
y_val = joblib.load('../Datasets/TwitterRF_SVM/y_val_youtube.pkl')
y_test = joblib.load('../Datasets/TwitterRF_SVM/y_test_youtube.pkl')

In [ ]:
# Normalizar os embeddings para o SVM
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
val_embeddings_scaled = scaler.transform(val_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

In [ ]:
# Avaliar o modelo no conjunto de teste e salvar previsões e rótulos
y_test_pred = best_svm_yt.predict(test_embeddings_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_class_report = classification_report(y_test, y_test_pred, target_names=['neutro', 'aprova', 'desaprova'])

print("Test Accuracy:", test_accuracy)
print("Test Classification Report:\n", test_class_report)


np.savez('../Datasets/YoutubeRF_SVM/test_predictions_and_labels_svm_yt_model_testdata_twitter.npz', predictions=y_test_pred, labels=y_test)


In [ ]:
# Carregar as previsões e rótulos salvos
data_svm = np.load('../Datasets/YoutubeRF_SVM/test_predictions_and_labels_svm_yt_model_testdata_twitter.npz')
predictions_test_svm = data_svm['predictions']
labels_test_svm = data_svm['labels']

In [ ]:
# Bootstrapping no conjunto de teste para RandomForest
ci_metrics_svm=bootstrap_test(predictions_test_svm, labels_test_svm, n_iterations=BOOTSTRAP_ITERATIONS)

In [ ]:
# Salvando os resultados do intervalo de confiança
results_rf = pd.DataFrame(ci_metrics_svm).T
results_rf.to_csv('../Datasets/YoutubeRF_SVM/bootstrap_results_svm_yt_model_data_twitter.csv')

print("Random Forest - Média, Desvio Padrão e Intervalos de Confiança das Métricas para as classes:")
print(results_rf)

# Análise

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Carregar os resultados do bootstrapping com cross
results_rf_tt_yt = pd.read_csv('../Datasets/Twitter vs Youtube/bootstrap_results_rf_tt_model_data_youtube.csv', index_col=0)
results_rf_yt_tt = pd.read_csv('../Datasets/Twitter vs Youtube/bootstrap_results_rf_yt_model_data_twitter.csv', index_col=0)
results_svm_tt_yt = pd.read_csv('../Datasets/Twitter vs Youtube/bootstrap_results_svm_tt_model_data_youtube.csv', index_col=0)
results_svm_yt_tt = pd.read_csv('../Datasets/Twitter vs Youtube/bootstrap_results_svm_yt_model_data_twitter.csv', index_col=0)
results_bert_tt_yt = pd.read_csv('../Datasets/Twitter vs Youtube/tt_model_yt_data_bootstrap_results.csv', index_col=0)
results_bert_yt_tt = pd.read_csv('../Datasets/Twitter vs Youtube/yt_model_tt_data_bootstrap_results.csv', index_col=0)

# Função para imprimir e comparar as métricas
def compare_results(results_list, labels):
    metrics = ['f1_class_0', 'f1_class_1', 'f1_class_2']
    comparison = []

    for metric in metrics:
        metric_data = {'Metric': metric}
        for results, label in zip(results_list, labels):
            mean = results.loc[metric, 'mean']
            error = mean - results.loc[metric, 'lower_95_ci']
            metric_data[f'{label}_mean'] = mean
            metric_data[f'{label}_error'] = error
        comparison.append(metric_data)

    comparison_df = pd.DataFrame(comparison)
    return comparison_df

# Comparar os resultados
results_list = [
    results_rf_tt_yt, results_rf_yt_tt,
    results_svm_tt_yt, results_svm_yt_tt,
    results_bert_tt_yt, results_bert_yt_tt
]
labels = [
    'RF_Twitter_on_YouTube', 'RF_YouTube_on_Twitter',
    'SVM_Twitter_on_YouTube', 'SVM_YouTube_on_Twitter',
    'BERT_Twitter_on_YouTube', 'BERT_YouTube_on_Twitter'
]

comparison_df = compare_results(results_list, labels)

# Função para plotar os gráficos com barras de erro
def plot_comparison(comparison_df):
    metrics = comparison_df['Metric']
    x = np.arange(len(metrics) * 2)  # Duas vezes o número de métricas (uma vez para Twitter, uma vez para YouTube)

    # Configurar as barras
    bar_width = 0.2
    opacity = 0.8

    # Criar a figura e os eixos
    fig, ax = plt.subplots(figsize=(8, 6))  # Aumentei o tamanho da figura para melhor visualização

    # Definir cores e rótulos
    colors = ['#2ca02c', '#ff7f0e', '#1f77b4']
    markers = ['o', 's', 'D']
    models = ['RF', 'SVM', 'BERT']
    datasets = ['Twitter', 'YouTube']
    classes = ['Neutro', 'Aprova', 'Desaprova']

    offset = 0
    for j, dataset in enumerate(datasets):
        for i, metric in enumerate(metrics):
            for k, model in enumerate(models):
                label = f'{model}_{dataset}_on_YouTube' if dataset == 'Twitter' else f'{model}_{dataset}_on_Twitter'
                mean = comparison_df.loc[i, f'{label}_mean']
                error = comparison_df.loc[i, f'{label}_error']
                ax.errorbar(i + offset, mean, yerr=error, fmt=markers[k],
                            color=colors[k], capsize=5, markersize=10, elinewidth=2, label=f'{model}' if i == 0 and j == 0 else "")
        offset += len(metrics)  # Move to the next set of columns for the other dataset

    # Adicionar rótulos e título
    ax.set_xlabel('Models and Classes', fontsize=14)
    ax.set_ylabel('F1-score', fontsize=14)
    ax.set_xticks(np.arange(len(metrics) * 2))
    ax.set_xticklabels(['Twitter Neutral', 'Twitter Approves', 'Twitter Disapproves',
                        'YouTube Neutral', 'YouTube Approves', 'YouTube Disapproves'], fontsize=14)
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), fontsize=12)

    # Padronizar valores do eixo y
    ax.set_ylim(0.4, 1)
    ax.set_ylim(0.4, 1)
    plt.xticks(rotation=10)
    plt.yticks(fontsize=14)
    plt.tight_layout()
    plt.savefig('../Datasets/Twitter vs Youtube/comparison_f1_cross_english.pdf', format='pdf')
    plt.show()

# Plotar a comparação
plot_comparison(comparison_df)

In [ ]:
['TN', 'TA', 'TD', 'YN', 'YA', 'YD'], 'TN(c)', 'TA(c)', 'TD(c)', 'YN(c)', 'YA(c)', 'YD(c)']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Carregar os resultados do bootstrapping sem cross
results_rf_tt = pd.read_csv('../Datasets/TwitterRF_SVM/bootstrap_results_rf.csv', index_col=0)
results_rf_yt = pd.read_csv('../Datasets/YoutubeRF_SVM/bootstrap_results_rf.csv', index_col=0)
results_svm_tt = pd.read_csv('../Datasets/TwitterRF_SVM/bootstrap_results_svm.csv', index_col=0)
results_svm_yt = pd.read_csv('../Datasets/YoutubeRF_SVM/bootstrap_results_svm.csv', index_col=0)
results_bert_tt = pd.read_csv('../Datasets/Twitter/bootstrap_results.csv', index_col=0)
results_bert_yt = pd.read_csv('../Datasets/YouTube/bootstrap_results.csv', index_col=0)

# Função para imprimir e comparar as métricas
def compare_results(results_list, labels):
    metrics = ['f1_class_0', 'f1_class_1', 'f1_class_2']
    comparison = []

    for metric in metrics:
        metric_data = {'Metric': metric}
        for results, label in zip(results_list, labels):
            mean = results.loc[metric, 'mean']
            error = mean - results.loc[metric, 'lower_95_ci']
            metric_data[f'{label}_mean'] = mean
            metric_data[f'{label}_error'] = error
        comparison.append(metric_data)

    comparison_df = pd.DataFrame(comparison)
    return comparison_df

# Comparar os resultados
results_list = [
    results_rf_tt, results_rf_yt,
    results_svm_tt, results_svm_yt,
    results_bert_tt, results_bert_yt
]
labels = [
    'RF_Twitter', 'RF_YouTube',
    'SVM_Twitter', 'SVM_YouTube',
    'BERT_Twitter', 'BERT_YouTube'
]

comparison_df = compare_results(results_list, labels)

# Função para plotar os gráficos com barras de erro
def plot_comparison(comparison_df):
    metrics = comparison_df['Metric']
    x = np.arange(len(metrics) * 2)  # Duas vezes o número de métricas (uma vez para Twitter, uma vez para YouTube)

    # Configurar as barras
    bar_width = 0.2
    opacity = 0.8

    # Criar a figura e os eixos
    fig, ax = plt.subplots(figsize=(4, 3))

    # Definir cores e rótulos
    colors = ['#2ca02c', '#ff7f0e', '#1f77b4']
    markers = ['o', 's', 'D']
    models = ['RF', 'SVM', 'BERT']
    datasets = ['Twitter', 'YouTube']
    classes = ['Neutro', 'Aprova', 'Desaprova']

    offset = 0
    for j, dataset in enumerate(datasets):
        for i, metric in enumerate(metrics):
            for k, model in enumerate(models):
                label = f'{model}_{dataset}'
                mean = comparison_df.loc[i, f'{label}_mean']
                error = comparison_df.loc[i, f'{label}_error']
                ax.errorbar(i + offset, mean, yerr=error, fmt=markers[k],
                            color=colors[k], capsize=5, markersize=10, elinewidth=2, label=f'{model}' if i == 0 and j == 0 else "")
        offset += len(metrics)  # Move to the next set of columns for the other dataset

    # Adicionar rótulos e título
    ax.set_xlabel('Models and Classes', fontsize=14)
    ax.set_ylabel('F1-score', fontsize=14)
    ax.set_xticks(np.arange(len(metrics) * 2))
    ax.set_xticklabels(['Twitter Neutral', 'Twitter Approves', 'Twitter Disapproves',
                        'YouTube Neutral', 'YouTube Approves', 'YouTube Disapproves'], fontsize=14)
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), fontsize=12)

    # Padronizar valores do eixo y
    ax.set_ylim(0.4, 1)
    plt.xticks(rotation=10)
    plt.yticks(fontsize=14)
    plt.tight_layout()
    plt.savefig('../Datasets/comparison_f1_nocross_english.pdf', format='pdf')
    plt.show()

# Plotar a comparação
plot_comparison(comparison_df)

In [ ]:
import pandas as pd

# Carregar os resultados do bootstrapping
results_twitter_on_youtube = pd.read_csv('../Datasets/Twitter vs Youtube/bootstrap_results_svm_tt_model_data_youtube.csv', index_col=0)
results_youtube_on_twitter = pd.read_csv('../Datasets/Twitter vs Youtube/bootstrap_results_svm_yt_model_data_twitter.csv', index_col=0)
results_rf_tt_yt = pd.read_csv('../Datasets/Twitter vs Youtube/bootstrap_results_rf_tt_model_data_youtube.csv')
results_rf_yt_tt = pd.read_csv('../Datasets/Twitter vs Youtube/bootstrap_results_rf_yt_model_data_twitter.csv')
results_svm_tt_yt = pd.read_csv('../Datasets/Twitter vs Youtube/bootstrap_results_svm_tt_model_data_youtube.csv')
results_svm_yt_tt = pd.read_csv('../Datasets/Twitter vs Youtube/bootstrap_results_svm_yt_model_data_twitter.csv')

# Função para imprimir e comparar as métricas
def compare_results(results1, results2, label1, label2):
    metrics = ['precision_class_0', 'recall_class_0', 'f1_class_0',
               'precision_class_1', 'recall_class_1', 'f1_class_1',
               'precision_class_2', 'recall_class_2', 'f1_class_2']

    comparison = []
    for metric in metrics:
        mean1 = results1.loc[metric, 'mean']
        std1 = results1.loc[metric, 'std']
        ci1 = (results1.loc[metric, 'lower_95_ci'], results1.loc[metric, 'upper_95_ci'])

        mean2 = results2.loc[metric, 'mean']
        std2 = results2.loc[metric, 'std']
        ci2 = (results2.loc[metric, 'lower_95_ci'], results2.loc[metric, 'upper_95_ci'])

        comparison.append({
            'Metric': metric,
            f'{label1}_mean': mean1,
            f'{label1}_std': std1,
            f'{label1}_95ci': ci1,
            f'{label2}_mean': mean2,
            f'{label2}_std': std2,
            f'{label2}_95ci': ci2
        })

    comparison_df = pd.DataFrame(comparison)
    print(comparison_df)
    return comparison_df

# Comparar os resultados
comparison_df = compare_results(results_twitter_on_youtube, results_youtube_on_twitter, 'SVMTwitter_on_YouTube', 'SVMYouTube_on_Twitter')

# Salvar a comparação em um arquivo CSV
# comparison_df.to_csv('./comparison_results.csv', index=False)


In [ ]:
comparison_df.to_csv('../Datasets/Twitter vs Youtube/comparison_results_svm.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
# Função para plotar os gráficos com barras de erro
def plot_comparison(comparison_df):
    metrics = comparison_df['Metric']
    x = range(len(metrics))

    twitter_means = comparison_df['SVMTwitter_on_YouTube_mean']
    twitter_errors = comparison_df['SVMTwitter_on_YouTube_std']

    youtube_means = comparison_df['SVMYouTube_on_Twitter_mean']
    youtube_errors = comparison_df['SVMYouTube_on_Twitter_std']

    plt.figure(figsize=(4, 3))  # Ajuste para 4x3 em polegadas (4x3)

    plt.errorbar(x, twitter_means, yerr=twitter_errors, fmt='o', capsize=5, label='Twitter Model on YouTube Data', color='blue')
    plt.errorbar(x, youtube_means, yerr=youtube_errors, fmt='o', capsize=5, label='YouTube Model on Twitter Data', color='green')

    plt.xticks(x, metrics, rotation=45, ha='right')

    plt.xlabel('Metrics')
    plt.ylabel('Scores')
    # plt.title('Cross-Platform Model Evaluation with Error Bars', fontsize=14)
    plt.legend(fontsize=4)
    plt.tight_layout()
    plt.show()

# Plotar a comparação
plot_comparison(comparison_df)

In [ ]:
# Função para plotar os gráficos com barras de erro
def plot_comparison(comparison_df):
    metrics = comparison_df['Metric']
    x = range(len(metrics))

    twitter_means = comparison_df['SVMTwitter_on_YouTube_mean']
    twitter_errors = comparison_df['SVMTwitter_on_YouTube_std']

    youtube_means = comparison_df['SVMYouTube_on_Twitter_mean']
    youtube_errors = comparison_df['SVMYouTube_on_Twitter_std']

    plt.figure(figsize=(12, 9))  # Ajuste para 4x3 em polegadas (4x3)

    plt.errorbar(x, twitter_means, yerr=twitter_errors, fmt='o', capsize=5, label='Twitter Model on YouTube Data', color='red')
    plt.errorbar(x, youtube_means, yerr=youtube_errors, fmt='o', capsize=5, label='YouTube Model on Twitter Data', color='steelblue')

    plt.xticks(x, metrics, rotation=45, ha='right', fontsize=12)  # Ajustando o tamanho da fonte dos ticks
    plt.xlabel('Metrics', fontsize=14)
    plt.ylabel('Scores', fontsize=14)
    # plt.title('Cross-Platform Model Evaluation with Error Bars', fontsize=16)
    plt.legend(fontsize=12)
    plt.tight_layout()
    plt.show()

# Plotar a comparação
plot_comparison(comparison_df)